In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time

def get_yahoo_crypto_data(symbol, period="max"):
    """
    Fetch full historical data from Yahoo Finance
    symbol: crypto symbol (e.g., 'BTC-USD')
    period: 'max' for all available data
    """
    try:
        # Create ticker object
        ticker = yf.Ticker(symbol)

        # Get historical data
        hist = ticker.history(period=period)

        if hist.empty:
            print(f"❌ No data found for {symbol}")
            return pd.DataFrame()

        # Reset index to get Date as column
        hist.reset_index(inplace=True)

        # Add crypto info
        hist['Symbol'] = symbol
        hist['Crypto'] = symbol.replace('-USD', '').replace('-USDT', '')

        # Clean up column names
        hist.columns = [col.replace(' ', '_') for col in hist.columns]

        return hist

    except Exception as e:
        print(f"❌ Error fetching {symbol}: {e}")
        return pd.DataFrame()

def get_coinbase_data_alternative(crypto_symbol):
    """
    Alternative method using different symbols
    """
    symbols_to_try = [
        f"{crypto_symbol}-USD",
        f"{crypto_symbol}-USDT",
        f"{crypto_symbol}USD=X"
    ]

    for symbol in symbols_to_try:
        print(f"   Trying symbol: {symbol}")
        df = get_yahoo_crypto_data(symbol)
        if not df.empty:
            return df, symbol

    return pd.DataFrame(), None

# Cryptocurrency symbols for Yahoo Finance
crypto_symbols = {
    'BTC': ['BTC-USD', 'Bitcoin'],
    'ETH': ['ETH-USD', 'Ethereum'],
    'BNB': ['BNB-USD', 'Binance Coin'],
    'SOL': ['SOL-USD', 'Solana'],
    'XRP': ['XRP-USD', 'Ripple'],
    'ADA': ['ADA-USD', 'Cardano'],
    'DOT': ['DOT-USD', 'Polkadot'],
    'LINK': ['LINK-USD', 'Chainlink'],
    'LTC': ['LTC-USD', 'Litecoin'],
    'BCH': ['BCH-USD', 'Bitcoin Cash']
}

print("=== 🚀 YAHOO FINANCE CRYPTO DATA FETCHER (100% FREE) ===")
print("Getting FULL historical data from launch to today!\n")

all_crypto_data = pd.DataFrame()
successful_downloads = []
failed_downloads = []

for crypto_code, (symbol, name) in crypto_symbols.items():
    print(f"📈 Fetching {name} ({symbol})...")

    df = get_yahoo_crypto_data(symbol, period="max")

    if not df.empty:
        # Add more info
        df['Crypto_Name'] = name
        df['Crypto_Code'] = crypto_code

        print(f"✅ SUCCESS! Got {len(df):,} days of data")
        print(f"   📅 From: {df['Date'].min().strftime('%Y-%m-%d')} to {df['Date'].max().strftime('%Y-%m-%d')}")
        print(f"   💰 Price range: ${df['Close'].min():.4f} - ${df['Close'].max():.2f}")

        all_crypto_data = pd.concat([all_crypto_data, df], ignore_index=True)
        successful_downloads.append(name)
    else:
        print(f"❌ Failed to get data for {name}")
        # Try alternative symbols
        print(f"   🔄 Trying alternative symbols...")
        alt_df, alt_symbol = get_coinbase_data_alternative(crypto_code)

        if not alt_df.empty:
            alt_df['Crypto_Name'] = name
            alt_df['Crypto_Code'] = crypto_code
            print(f"✅ SUCCESS with {alt_symbol}! Got {len(alt_df):,} days")
            all_crypto_data = pd.concat([all_crypto_data, alt_df], ignore_index=True)
            successful_downloads.append(f"{name} (alt)")
        else:
            failed_downloads.append(name)

    print()  # Empty line for readability
    time.sleep(1)  # Be nice to Yahoo servers

# Save all data
if not all_crypto_data.empty:
    # Sort by crypto and date
    all_crypto_data = all_crypto_data.sort_values(['Crypto_Code', 'Date']).reset_index(drop=True)

    # Save to CSV
    filename = 'crypto_historical_yahoo_finance.csv'
    all_crypto_data.to_csv(filename, index=False)

    print("🎉 =" * 50)
    print("🎉 DATA SUCCESSFULLY DOWNLOADED!")
    print("🎉 =" * 50)
    print(f"📁 File saved: {filename}")
    print(f"📊 Total records: {len(all_crypto_data):,}")
    print(f"💱 Cryptocurrencies: {len(all_crypto_data['Crypto_Code'].unique())}")

    print(f"\n✅ SUCCESSFUL DOWNLOADS ({len(successful_downloads)}):")
    for crypto in successful_downloads:
        print(f"   • {crypto}")

    if failed_downloads:
        print(f"\n❌ FAILED DOWNLOADS ({len(failed_downloads)}):")
        for crypto in failed_downloads:
            print(f"   • {crypto}")

    # Show detailed summary
    print(f"\n📈 DETAILED SUMMARY:")
    summary = all_crypto_data.groupby(['Crypto_Code', 'Crypto_Name']).agg({
        'Date': ['min', 'max', 'count'],
        'Close': ['min', 'max', 'first', 'last']
    }).round(6)

    print("=" * 80)
    for crypto_code in all_crypto_data['Crypto_Code'].unique():
        crypto_data = all_crypto_data[all_crypto_data['Crypto_Code'] == crypto_code]
        crypto_name = crypto_data['Crypto_Name'].iloc[0]

        start_date = crypto_data['Date'].min().strftime('%Y-%m-%d')
        end_date = crypto_data['Date'].max().strftime('%Y-%m-%d')
        days = len(crypto_data)

        start_price = crypto_data.loc[crypto_data['Date'].idxmin(), 'Close']
        current_price = crypto_data.loc[crypto_data['Date'].idxmax(), 'Close']

        min_price = crypto_data['Close'].min()
        max_price = crypto_data['Close'].max()

        # Calculate total returns
        total_return = ((current_price - start_price) / start_price) * 100

        print(f"\n🪙 {crypto_name} ({crypto_code}):")
        print(f"   📅 Period: {start_date} to {end_date} ({days:,} days)")
        print(f"   💰 Price: ${start_price:.6f} → ${current_price:.2f}")
        print(f"   📊 Range: ${min_price:.6f} - ${max_price:.2f}")
        print(f"   📈 Total Return: {total_return:+.1f}%")

    print("\n💡 TIP: You now have COMPLETE historical data for analysis!")
    print("💡 This includes: Open, High, Low, Close, Volume for each day")

else:
    print("❌ No data was successfully downloaded.")
    print("💡 Try checking your internet connection or try again later.")

print(f"\n🔍 COLUMNS IN DATASET:")
if not all_crypto_data.empty:
    print(f"   {list(all_crypto_data.columns)}")
else:
    print("   No data available to show columns.")

=== 🚀 YAHOO FINANCE CRYPTO DATA FETCHER (100% FREE) ===
Getting FULL historical data from launch to today!

📈 Fetching Bitcoin (BTC-USD)...
✅ SUCCESS! Got 3,923 days of data
   📅 From: 2014-09-17 to 2025-06-14
   💰 Price range: $178.1030 - $111673.28

📈 Fetching Ethereum (ETH-USD)...
✅ SUCCESS! Got 2,774 days of data
   📅 From: 2017-11-09 to 2025-06-14
   💰 Price range: $84.3083 - $4812.09

📈 Fetching Binance Coin (BNB-USD)...
✅ SUCCESS! Got 2,774 days of data
   📅 From: 2017-11-09 to 2025-06-14
   💰 Price range: $1.5104 - $750.27

📈 Fetching Solana (SOL-USD)...
✅ SUCCESS! Got 1,891 days of data
   📅 From: 2020-04-10 to 2025-06-14
   💰 Price range: $0.5153 - $261.87

📈 Fetching Ripple (XRP-USD)...
✅ SUCCESS! Got 2,774 days of data
   📅 From: 2017-11-09 to 2025-06-14
   💰 Price range: $0.1396 - $3.38

📈 Fetching Cardano (ADA-USD)...
✅ SUCCESS! Got 2,774 days of data
   📅 From: 2017-11-09 to 2025-06-14
   💰 Price range: $0.0240 - $2.97

📈 Fetching Polkadot (DOT-USD)...
✅ SUCCESS! Got 1,7